In [1]:
import h5py
import pydub # faster than librosa
import numpy      as np
import pandas     as pd
import time

from pytorch.coughvid_dataset import CoughvidDataset
from pytorch.test_coughvid_dataset import TestCoughvidDataset
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.models import resnet50

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy

In [2]:
dir = 'C:/COUGHVID_public_dataset/public_dataset/'

full_dataset = CoughvidDataset(dir, 'metadata_compiled.csv', get_features=True)

dataframe = full_dataset.dataframe

minority_class_count = len(dataframe[dataframe['status']==2])

print(f'{minority_class_count} samples in the minority class.')

Mask file C:/COUGHVID_public_dataset/public_dataset/ does not exist. Calculating masks on the fly.
8997 records ready to load across 2 groups.
699 samples in the minority class.


c:\python39\lib\site-packages\torchaudio\functional\functional.py:594: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [3]:
sample_dataset = CoughvidDataset(dir, 'metadata_compiled.csv', get_features=True, samples_per_class=100)


Mask file C:/COUGHVID_public_dataset/public_dataset/ does not exist. Calculating masks on the fly.
200 records ready to load across 2 groups.


In [11]:
# split data into training and test samples
train_indices, test_indices = train_test_split(np.arange(0,len(sample_dataset)-1), test_size=0.25)

batch_size  = 1
num_workers = 6

train_loader  = DataLoader(sample_dataset, 
                           num_workers=num_workers,
                           sampler=SubsetRandomSampler(train_indices)
                          )

test_loader   = DataLoader(sample_dataset, 
                           num_workers=num_workers,
                           sampler=SubsetRandomSampler(test_indices)
                          )

dataloaders = {
    "train": train_loader,
    "test": test_loader
}

In [12]:
# load model and change output shape for binary prediction
model = resnet50()

model.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=2048,
        out_features=1
    ),
    torch.nn.Sigmoid()
)



#model.conv1 = nn.Conv2d(475, 64, kernel_size=7, stride=2, padding=3,bias=False)


In [10]:
'''RESNET training code adapted from https://www.kaggle.com/gxkok21/resnet50-with-pytorch'''

#if torch.cuda.is_available():
#    model = model.cuda()
#    print('Using GPU.')

optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.BCELoss()

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

EPOCHS = 5

for i in range(EPOCHS):
    for phase in ["train", "test"]:
        if phase == "train":
            model.train()
        else:
            model.eval()
        
        samples = 0
        loss_sum = 0
        correct_sum = 0
        for j, batch in enumerate(dataloaders[phase]):
            X,labels = batch
            print(labels)
            X = np.array(X)
            print(X.shape)
            if torch.cuda.is_available():
                X = X.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                y = model(X)
                loss = criterion(
                    y, 
                    labels[0].float()
                )

                if phase == "train":
                    loss.backward()
                    optimizer.step()
                    
                loss_sum += loss.item() * X.shape[0] # We need to multiple by batch size as loss is the mean loss of the samples in the batch
                samples += X.shape[0]
                num_corrects = torch.sum((y >= 0.5).float() == labels[0].float())
                correct_sum += num_corrects
                
                # Print batch statistics every 50 batches
                if j % 50 == 49 and phase == "train":
                    print("{}:{} - loss: {}, acc: {}".format(
                        i + 1, 
                        j + 1, 
                        float(loss_sum) / float(samples), 
                        float(correct_sum) / float(samples)
                    ))
                
        # Print epoch statistics
        epoch_acc = float(correct_sum) / float(samples)
        epoch_loss = float(loss_sum) / float(samples)
        print("epoch: {} - {} loss: {}, {} acc: {}".format(i + 1, phase, epoch_loss, phase, epoch_acc))
        
        # Deep copy the model
        if phase == "test" and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "resnet50.pth")

tensor([0], dtype=torch.int32)
(19, 25)


C:\Users\Filip\AppData\Local\Temp/ipykernel_4340/804223748.py:26: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X = np.array(X)
C:\Users\Filip\AppData\Local\Temp/ipykernel_4340/804223748.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m, [31;1mParameter[0m, [31;1mNoneType[0m, [31;1mtuple[0m, [31;1mtuple[0m, [31;1mtuple[0m, [32;1mint[0m)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m, [31;1mParameter[0m, [31;1mNoneType[0m, [31;1mtuple[0m, [31;1mtuple[0m, [31;1mtuple[0m, [32;1mint[0m)
